In [1]:
import sys
print(sys.version) #should be <3.12

3.10.4 (v3.10.4:9d38120e33, Mar 23 2022, 17:29:05) [Clang 13.0.0 (clang-1300.0.29.30)]


In [2]:
import pandas as pd, numpy as np
from pprint import pprint

from lime.lime_text import LimeTextExplainer
from lime.lime_tabular import LimeTabularExplainer
from sklearn.pipeline import make_pipeline

import torch
import tensorflow as tf
import tensorflow.keras.backend as K
from transformers import *
import tokenizers
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('TF version',tf.__version__)

2024-02-01 18:27:26.823617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-01 18:27:38.180712: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TF version 2.11.0


### Load model

In [20]:
config = RobertaConfig.from_pretrained('roBERTaFiles/config-roberta-base.json')
bert_model = TFRobertaModel.from_pretrained('roBERTaFiles/pretrained-roberta-base.h5',config=config)
bert_model.summary()
new_model = tf.keras.models.load_model('v0-roberta-0l.h5',custom_objects={"TFRobertaModel": bert_model})

# Show the model architecture
new_model.summary()

loading configuration file roBERTaFiles/config-roberta-base.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "eos_token_ids": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": null,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



OSError: roBERTaFiles/pretrained-roberta-base.h5 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [4]:
PATH = 'roBERTaFiles'
DISPLAY=1
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
test = pd.read_csv('tweet-sentiment-extraction/test.csv').fillna('')

# tokenizer
#def tokenizer(sentiment_id=sentiment_id, test = test):
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab=PATH+'/vocab-roberta-base.json', 
    merges=PATH+'/merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)

MAX_LEN = 96

# test tweets
ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]): 
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask_t[k,:len(enc.ids)+5] = 1

pprint(input_ids_t[0])#[input_ids_t,attention_mask_t,token_type_ids_t]

array([    0,    94,  1852,     9,     5,   183,  2054,   640, 17137,
         405, 19017,     4,   175,    73,  4111,  5841,   298,     2,
           2,  7974,     2,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1], dtype=int32)


In [5]:
#len(tokenizer()[0])

In [6]:
predictions = new_model.predict([input_ids_t,attention_mask_t,token_type_ids_t])

111/111 [==============================] - 777s 7s/step


In [7]:
from shap import DeepExplainer, force_plot, initjs

In [8]:
# use KernelShap to get explaination
#input =[input_ids_t,attention_mask_t,token_type_ids_t]
#exp = DeepExplainer(new_model, input)
#shap_values = exp.shap_values(input) #here instance_id
# plot shapely values
#force_plot(exp.expected_value, shap_values, X_test.iloc[instance_id])

In [19]:
oof_start = predictions[0]
oof_end = predictions[1]

### LIME
compare only with roberta sentiment part

or with start/end

In [11]:
print(test['text'][0])

Last session of the day  http://twitpic.com/67ezh


In [61]:
input_s = pd.read_csv('tweet-sentiment-extraction/test.csv')
input_s.head()
categorical_features = [i for i, col in enumerate(input_s) if np.unique(col).size < 10]

input_t = pd.DataFrame({'i':input_ids_t.tolist(),'m':attention_mask_t.tolist(),'tt':token_type_ids_t.tolist()})
input_t['m'] = [tmp.index(1) for tmp in input_t['m']]
input_t[:]
#print(input_t['m'])

,i,m,tt
0,"[0, 94, 1852, 9, 5, 183, 2054, 640, 17137, 405...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 1481, 260, 43673, 16, 67, 269, 3571, 36, 5...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 7306, 478, 4342, 261, 5150, 5378, 42134, 1...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 1372, 741, 1208, 328, 2, 2, 1313, 2, 1, 1,...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 2054, 640, 17137, 405, 19017, 4, 175, 73, ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(use_idf=True)
tfidf_id = tfidf.fit_transform(input_s['textID']) 
tfidf_t = tfidf.fit_transform(input_s['text']) 

In [35]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(tfidf, new_model)

In [41]:
explainer = LimeTextExplainer(class_names = sentiment_id.keys())
# (input_s.iloc[0]['textID'],input_s.iloc[0]['text'],input_s.iloc[0]['sentiment'])
exp = explainer.explain_instance(input_s.iloc[0]['text'],
                                 bert_model.predict, num_features=3 )

ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filter

In [45]:
explainer = LimeTabularExplainer(np.array(input_t),class_names = sentiment_id.keys(),
                                 training_labels = input_t['tt'],
                                 categorical_features=categorical_features, 
                                 discretizer= 'entropy')

exp = explainer.explain_instance(input_s.iloc[[0]],new_model.predict, num_features=3 )

#exp.show_in_notebook(text=False)

lime_df = pd.DataFrame(exp.as_list(),columns=['Feature','Contribution'])
lime_df.head()

ValueError: setting an array element with a sequence.

### TCAV

- first create lexicon for pos/neg/neutral sentiments
- then use TCAV (captum) for projection on sentiments (concepts), probably after roBERTa layer


In [ ]:
import torchtext
from torchtext import vocab
from torchtext.datasets import LanguageModelingDataset

import tensorflow as tf

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, IterableDataset

import spacy
import en_core_web_sm

import numpy as np
import os
import matplotlib.pyplot as plt

from scipy.stats import ttest_ind

#.... Captum imports..................
from captum.concept import TCAV
from captum.concept import Concept
from captum.concept._utils.common import concepts_to_str
from captum.concept._utils.data_iterator import dataset_to_dataloader, CustomIterableDataset

nlp = en_core_web_sm.load()

In [ ]:
#load VADER (https://github.com/cjhutto/vaderSentiment?tab=readme-ov-file#resources-and-dataset-descriptions) lexicon
lexicon = pd.read_csv('tweet-sentiment-extraction/vader_lexicon.txt', sep = "\t", header = None)

# separate pos (>1), neg (<-1&<0), neu(>-1 & <1) based on polarity-scores (column 1)
def convert_labels(elem):
    if elem > 1.0:
        return 'positive'
    elif elem > -1.0 and elem < 1.0:
        return 'neutral'
    else:
        return 'negative'
            

lexicon[1] = lexicon[1].apply(lambda x: convert_labels(x))

# create lexicon files for each sentiment
lexicon = lexicon.drop(columns= [2,3])

df_pos = lexicon[lexicon[1] == 'positive']
df_neg = lexicon[lexicon[1] == 'negative']
df_neu = lexicon[lexicon[1] == 'neutral']

df_pos.to_csv('tweet-sentiment-extraction/positive.csv', index=False)
df_neg.to_csv('tweet-sentiment-extraction/negative.csv', index=False)
df_neu.to_csv('tweet-sentiment-extraction/neutral.csv', index=False)

In [ ]:
# make data field and load data
def spacy_tok(x):
    return [tok.text for tok in nlp.tokenizer(x)]

TEXT = torchtext.data.Field(lower=True, tokenizer_language='en_core_web_sm', tokenize = spacy_tok)
Label = torchtext.data.LabelField(dtype = torch.float)

ds_pos = LanguageModelingDataset("tweet-sentiment-extraction/positive.csv", TEXT)
ds_neut = LanguageModelingDataset("tweet-sentiment-extraction/neutral.csv", TEXT)
ds_neg = LanguageModelingDataset("tweet-sentiment-extraction/negative.csv", TEXT)

TEXT.build_vocab(ds_pos,ds_neut,ds_neg)

# load glove
MAX_VOCAB_SIZE = 25_000
loaded_vectors= torchtext.vocab.Vectors('tweet-sentiment-extraction/glove.6B.50d.txt')

#TEXT.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))
TEXT.vocab.set_vectors(stoi=loaded_vectors.stoi, vectors=loaded_vectors.vectors, dim=loaded_vectors.dim)
print('Vocabulary Size: ', len(TEXT.vocab))

In [ ]:
def get_tensor_from_filename(filename):
    ds = torchtext.data.TabularDataset(path=filename,
                                       fields=[('text', torchtext.data.Field()),
                                               ('label', torchtext.data.Field())],
                                       format='csv')
    const_len = 7
    for concept in ds:
        concept.text = concept.text[:const_len]
        concept.text += ['pad'] * max(0, const_len - len(concept.text))
        text_indices = torch.tensor([TEXT.vocab.stoi[t] for t in concept.text], device=device)
        yield text_indices
        
        
def assemble_concept(name, id, concepts_path):
    dataset = CustomIterableDataset(get_tensor_from_filename, concepts_path)
    concept_iter = dataset_to_dataloader(dataset, batch_size=1)
    return Concept(id=id, name=name, data_iter=concept_iter)


def print_concept_sample(concept_iter):
    cnt = 0
    max_print = 10
    item = next(concept_iter)
    while cnt < max_print and item is not None:
        print(' '.join([TEXT.vocab.itos[item_elem] for item_elem in item[0]]))
        item = next(concept_iter)
        cnt += 1

In [ ]:
# create concepts: match glove embeddings to words in lexicon
neut = assemble_concept('neutral', 0, concepts_path="tweet-sentiment-extraction/neutral.csv")
pos = assemble_concept('positive', 1, concepts_path="tweet-sentiment-extraction/positive.csv")
neg = assemble_concept('negative', 2, concepts_path="tweet-sentiment-extraction/negative.csv")

In [ ]:
print_concept_sample(iter(pos.data_iter))

In [ ]:
# create multiple neutral concepts to test 
# repeat with neg and pos
neutral_concept = assemble_concept('neutral', 0, concepts_path="data/tcav/text-sensitivity/neutral.csv")
neutral_concept4 = assemble_concept('neutral4', 3, concepts_path="data/tcav/text-sensitivity/neutral4.csv")
neutral_concept5 = assemble_concept('neutral5', 4, concepts_path="data/tcav/text-sensitivity/neutral5.csv")

In [ ]:
# assemble sets to compare
# repeat with diff layers
# TCAV trains a model for each pair, and estimates tcav scores for each experimental set in given input layers.
# score indicates the importance of a concept in a given layer (high = inportant)
experimental_sets=[[pos, neut],
                  [pos, neg]]

tcav = TCAV(new_model, layers=['tf_roberta_model_1', 'conv1d'])

In [ ]:
# convert concepts to tensors
def covert_text_to_tensor(input_texts):
    input_tensors = []
    for input_text in input_texts:
        input_tensor = torch.tensor([TEXT.vocab.stoi[tok.text] for \
                                     tok in nlp.tokenizer(input_text)], device=device).unsqueeze(0)
        input_tensors.append(input_tensor)
    return torch.cat(input_tensors)

# show tcav scores 
def extract_scores(interpretations, layer_name, score_type, idx):
    return [interpretations[key][layer_name][score_type][idx].item() for key in interpretations.keys()]

In [ ]:
# test tcav 
pos_input_texts = ["It was a fantastic play ! pad", "A terrific film so far ! pad", "We loved that show ! pad pad"]
pos_input_text_indices = covert_text_to_tensor(pos_input_texts)

positive_interpretations = tcav.interpret(pos_input_text_indices, experimental_sets=experimental_sets)

#### visualize tcav

In [ ]:
def format_float(f):
    return float('{:.3f}'.format(f) if abs(f) >= 0.0005 else '{:.3e}'.format(f))

def plot_tcav_scores(experimental_sets, tcav_scores, layers = ['convs.2'], score_type='sign_count'):
    fig, ax = plt.subplots(1, len(experimental_sets), figsize = (25, 7))

    barWidth = 1 / (len(experimental_sets[0]) + 1)

    for idx_es, concepts in enumerate(experimental_sets):
        concepts = experimental_sets[idx_es]
        concepts_key = concepts_to_str(concepts)
        
        layers = tcav_scores[concepts_key].keys()
        pos = [np.arange(len(layers))]
        for i in range(1, len(concepts)):
            pos.append([(x + barWidth) for x in pos[i-1]])
        _ax = (ax[idx_es] if len(experimental_sets) > 1 else ax)
        for i in range(len(concepts)):
            val = [format_float(scores[score_type][i]) for layer, scores in tcav_scores[concepts_key].items()]
            _ax.bar(pos[i], val, width=barWidth, edgecolor='white', label=concepts[i].name)

        # Add xticks on the middle of the group bars
        _ax.set_xlabel('Set {}'.format(str(idx_es)), fontweight='bold', fontsize=16)
        _ax.set_xticks([r + 0.3 * barWidth for r in range(len(layers))])
        _ax.set_xticklabels(layers, fontsize=16)

        # Create legend & Show graphic
        _ax.legend(fontsize=16)

    plt.show()

In [ ]:
# change inputs!!
plot_tcav_scores(experimental_sets, positive_interpretations, ['tf_roberta_model_1', 'conv1d'], score_type='sign_count')